# Exercise: a bit more data processing
- 나중에 유용하게 사용할 열을 추가하고
- 다른 열의 값을 변경하고
- 특정 열을 완전히 삭제

In [ ]:
import pandas as pd
import time
import ast # ast 모듈은 Python 코드 구문 트리를 파싱하고 분석하는데 사용

# Initial column processing
- 위도 및 경도 삭제
- Hotel_Address 값을 변경 (주소에 도시와 국가가 같은 경우 도시와 국가로만 변경)

- 다음은 데이터 집합에 있는 고유한 도시와 국가:
    - Amsterdam, Netherlands
    - Barcelona, Spain
    - London, United Kingdom
    - Milan, Italy
    - Paris, France
    - Vienna, Austria

In [ ]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 
    else:
        return row.Hotel_Address
    

In [ ]:
# Load the hotel reviews from CSV
start = time.time()
df = pd.read_csv('./Hotel_Reviews.csv')


In [ ]:
# dropping columns we will not use:
df.drop(["lat", "lng"], axis = 1, inplace=True)


In [ ]:
# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)


In [ ]:
# The sum of the value_counts() should add up to the total number of reviews
print(df["Hotel_Address"].value_counts())

In [ ]:
# Query country level data:
# df 데이터프레임을 사용하여 호텔 주소별 고유한 호텔 이름의 개수를 집계하고 결과를 표시
# df.groupby("Hotel_Address"): df 데이터프레임을 Hotel_Address 열을 기준으로 그룹화
# .agg({"Hotel_Name": "nunique"}): 각 그룹에 대해 Hotel_Name 열의 고유한 값의 개수를 계산
# display(...): 결과를 화면에 표시

display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

In [ ]:
# Process Hotel Meta-review columns

# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"], axis = 1, inplace=True)

# Replace `Total_Number_of_Reviews` and `Average_Score` with our own calculated values
df['Total_Number_of_Reviews'] = df.groupby('Hotel_Name')['Hotel_Name'].transform('count')
df['Average_Score'] = round(df.groupby('Hotel_Name')['Reviewer_Score'].transform('mean'), 1)

In [ ]:
# Drop Review_Total_Negative_Word_Counts, Review_Total_Positive_Word_Counts, Review_Date and days_since_review
# Drop Total_Number_of_Reviews_Reviewer_Has_Given

df.drop(["Review_Date", "Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "days_since_review", "Total_Number_of_Reviews_Reviewer_Has_Given"], axis = 1, inplace=True)

# Tag columns
- 태그 열은 열에 저장된 목록(텍스트 형식)인데 문제가 있음
    - 이 열의 하위 섹션의 순서와 개수가 항상 같지는 않음
    - 515,000개의 행과 1,427개의 호텔이 있고 리뷰어가 선택할 수 있는 옵션이 각각 조금씩 다르기 때문에 우리가 관심을 가질 수 있는 정확한 문구를 식별하기는 어려움
    - 텍스트를 스캔하여 가장 일반적인 문구를 찾아서 그 수를 세어볼 수 있음

- 우리는 다중 단어 구문(예: busiess trip)에 관심이 있음
    - 탐색적 데이터 분석이 유용한데, ['business trip ', 'solo traverler ', 'single room ', '5 nights ', 'submitted from mobile device'] 등의 태그 샘플을 보면 처리해야 하는 작업을 줄일 수 있는지 확인할 수 있음

# Filtering tags
- 데이터 세트의 처리 목표는 최고의 호텔을 선택하는 데 도움이 되는 감정과 열을 추가하는 것
    - 여행 유형은 관련성이 있으므로 그대로 유지
    - 게스트 그룹의 유형이 중요하며, 해당 유형이 유지
    - 게스트가 투숙한 객실, 스위트 또는 스튜디오 유형은 관련이 없음 (모든 호텔은 기본적으로 동일한 객실을 보유함).
    - 후기를 제출한 기기는 관련이 없음
    - 리뷰 작성자의 투숙 기간이 길수록 호텔을 더 좋아한다고 판단할 수 있지만, 이는 지나친 추측이며 관련성이 없을 수 있음
    - 결론: 여행 유형과 게스트 그룹 태그만 유지하고 나머지는 제거

In [ ]:
# 대괄호와 따옴표를 제거

# remove opening and closing brackets)
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

In [ ]:
print (df["Tags"])

# Each tag becomes something like: Business trip, Solo traveler, Single Room, Stayed 5 nights, Submitted from a mobile device

# 다음 문제
- 어떤 리뷰는 필드가 5개, 일부는 3개, 일부는 6개
- 이는 데이터 집합이 생성된 방법의 결과이며 수정하기 어려움
- 각 필드를 차지하는 tag (문구)의 빈도 수를 계산하는 것은 각 리뷰에서 문구의 순서가 다르기 때문에 count가 잘못될 수 있으며, 당연히 있어야할 tag이 없을 수도 있음
- 각 tag는 여러 단어로 이루어져 있지만 쉼표로 구분되어 있음
- 각 tag를 순서에 해당하는 필드에 삽입하여 임시 열 6개를 만들 수 있음
- 그런 다음 6개의 필드를 하나의 레코드로 병합하고 결과 필드에서 value_counts() 메서드를 실행
- 출력하면 2,428개의 고유 태그가 있음을 알 수 있음

In [ ]:
# Now split the strings into a list
tag_list_df = df.Tags.str.split(',', expand = True)

In [ ]:
# Remove leading and trailing spaces
df["Tag_1"] = tag_list_df[0].str.strip()
df["Tag_2"] = tag_list_df[1].str.strip()
df["Tag_3"] = tag_list_df[2].str.strip()
df["Tag_4"] = tag_list_df[3].str.strip()
df["Tag_5"] = tag_list_df[4].str.strip()
df["Tag_6"] = tag_list_df[5].str.strip()

In [ ]:
# Merge the 6 columns into one with melt
df_tags = df.melt(value_vars=["Tag_1", "Tag_2", "Tag_3", "Tag_4", "Tag_5", "Tag_6"])

In [ ]:
# Submitted from a mobile device됨과 같은 일반적인 태그 중 일부는 쓸모가 없으므로 제거
# length of stay 태그 제거
# 객실, 스위트, 스튜디오, 아파트 등 매우 다양한 종류의 숙소가 있지만 거의 같은 의미이며 목표와 관련이 없으므로 고려 대상에서 제외

# Get the value counts
tag_vc = df_tags.value.value_counts()
# print(tag_vc)
print("The shape of the tags with no filtering:", str(df_tags.shape))
# Drop rooms, suites, and length of stay, mobile device and anything with less count than a 1000
df_tags = df_tags[~df_tags.value.str.contains("Standard|room|Stayed|device|Beds|Suite|Studio|King|Superior|Double", na=False, case=False)]
tag_vc = df_tags.value.value_counts().reset_index(name="count").query("count > 1000")
# Print the top 10 (there should only be 9 and we'll use these in the filtering section)
print(tag_vc[:10])

In [ ]:
# 마지막 단계는 이러한 각 태그에 관한 새 필드를 만드는 것
# 다음 모든 리뷰 레코드에서 태그 필드가 새 필드 중 하나와 일치하면 1을 추가하고, 일치하지 않으면 0을 추가
# 최종 결과는 비즈니스와 레저, 반려동물 동반 등의 이유로 이 호텔을 선택한 리뷰어의 수(총합)
# 이는 호텔을 추천할 때 유용한 정보로 활용할 수 있음

# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends, 
# Family with young children, Family with older children, With a pet
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)


In [ ]:
# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r'./Hotel_Reviews_Filtered.csv', index = False)
end = time.time()
print("Filtering took " + str(round(end - start, 2)) + " seconds")


# Sentiment Analysis Operations
- Apply sentiment analysis to the review columns and save the results in a dataset.

# Exercise: load and save the filtered data

In [ ]:
%pip install nltk
import time
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer # nltk의 VADER 감정 분석기를 가져옮
nltk.download('vader_lexicon') # VADER 감정 분석기에 필요한 어휘 자료를 다운로드

# Load the filtered hotel reviews from CSV
df = pd.read_csv('./Hotel_Reviews_Filtered.csv')

# Removing stop words
- 부정적 및 긍정적 리뷰 열에 대해 감성 분석 기능을 실행하는 경우 시간이 오래 걸릴 수 있음
- 문장의 감성을 바꾸지 않는 일반적인 영어 단어인 stop words를 제거할 수 있음
- 이러한 단어를 제거하면 감성어 분석이 더 빠르게 실행되지만 정확도가 떨어지지는 않음
    - 가장 긴 부정적 리뷰는 395단어였지만, 중지 단어를 제거한 후에는 195단어로 줄었습니다.

In [ ]:
nltk.download('stopwords')

# Remove stop words - can be slow for a lot of text!
# Ryan Han (ryanxjhan on Kaggle) has a great post measuring performance of different stop words removal approaches
# https://www.kaggle.com/ryanxjhan/fast-stop-words-removal # using the approach that Ryan recommends
start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text

# Remove the stop words from both columns
df.Negative_Review = df.Negative_Review.apply(remove_stopwords)   
df.Positive_Review = df.Positive_Review.apply(remove_stopwords)

# Performing sentiment analysis

- 부정적인 리뷰와 긍정적인 리뷰 열 모두에 대한 감성 분석을 계산하고 결과를 2개의 새 열에 저장
- 감정 테스트는 동일한 리뷰에 대한 리뷰어의 점수와 비교하는 것
    - 예를 들어, 부정적인 리뷰의 감성이 1(매우 긍정적인 감성)이고 긍정적인 리뷰의 감성이 1인데 리뷰어가 해당 호텔에 가장 낮은 점수를 준 경우, 리뷰 텍스트가 점수와 일치하지 않거나 감성 분석기가 감성을 제대로 인식하지 못했을 수 있음
    - 예를 들어 "당연히 난방이 안 되는 방에서 자는 것이 좋았다"라는 극도로 비꼬는 리뷰가 있는데, 이를 읽는 사람은 비꼬는 말이라는 것을 알 수 있음에도 불구하고 감성 분석기는 이를 긍정적인 감성이라고 생각하는 등, 일부 감성 점수는 완전히 틀릴 수 있으며, 이는 설명할 수 있는 경우가 대부분

- NLTK는 학습할 수 있는 다양한 감성 분석기를 제공하며, 사용자는 이를 대체하여 감성이 어느 정도 정확한지 확인할 수 있음
    - 여기서는 VADER 감정 분석기를 사용

In [ ]:
vader_sentiment = SentimentIntensityAnalyzer() # VADER 감정 분석기를 초기화

# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review):    
    if review == "No Negative" or review == "No Positive": # 리뷰가 "No Negative" 또는 "No Positive"인 경우
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]  # 리뷰가 텍스트인 경우 VADER 감정 분석기를 사용하여 감정 점수(컴파운드 스코어)를 계산하고 반환

In [ ]:
# Add a negative sentiment and positive sentiment column
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

In [ ]:
# Print of the results and see if the sentiment matches the review

df = df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(df[["Negative_Review", "Negative_Sentiment"]])
df = df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(df[["Positive_Review", "Positive_Sentiment"]])

In [ ]:
# Reorder the columns (This is cosmetic, but to make it easier to explore the data later)
df = df.reindex(["Hotel_Name", "Hotel_Address", "Total_Number_of_Reviews", "Average_Score", "Reviewer_Score", "Negative_Sentiment", "Positive_Sentiment", "Reviewer_Nationality", "Leisure_trip", "Couple", "Solo_traveler", "Business_trip", "Group", "Family_with_young_children", "Family_with_older_children", "With_a_pet", "Negative_Review", "Positive_Review"], axis=1)

print("Saving results to Hotel_Reviews_NLP.csv")
df.to_csv(r"./Hotel_Reviews_NLP.csv", index = False)